### I now have tables of precip, snowfall, snowpack, weather, and avy dangers. now to finally put it all together, hopefully!

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## There are many weather stations across the Steven Pass area. 
### I will choose the tye-mill station for it is the closest to the Brooks elevation for the precipitaion data from the Brooks station
### Therefore, I will be predicting the near treeline avalanche danger

In [2]:
tyemill = pd.read_csv("csv_files/StevensPass-TyeMill_5180_feet_2019.csv")

In [3]:
tyemill.head()

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.)
0,2019-05-14 07:00,12.80,37.21,0.000,4.660,20.34,250.3
1,2019-05-14 06:00,12.84,44.51,0.000,4.046,10.11,278.6
2,2019-05-14 05:00,12.94,43.41,0.723,7.701,15.02,272.7
3,2019-05-14 04:00,13.57,43.25,4.340,10.050,18.87,271.9
4,2019-05-14 03:00,12.64,42.71,3.288,9.440,17.34,275.2


## I want to extract the date without the time of day so i can match this to my avalanche table

In [4]:
type(tyemill['Date/Time (PST)'][0])

str

In [5]:
tyemill['date'] = tyemill['Date/Time (PST)'].apply(lambda x: x.split(' ')[0])


In [6]:
tyemill.head()

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date
0,2019-05-14 07:00,12.80,37.21,0.000,4.660,20.34,250.3,2019-05-14
1,2019-05-14 06:00,12.84,44.51,0.000,4.046,10.11,278.6,2019-05-14
2,2019-05-14 05:00,12.94,43.41,0.723,7.701,15.02,272.7,2019-05-14
3,2019-05-14 04:00,13.57,43.25,4.340,10.050,18.87,271.9,2019-05-14
4,2019-05-14 03:00,12.64,42.71,3.288,9.440,17.34,275.2,2019-05-14


In [7]:
tyemill.shape

(3200, 8)

In [8]:
tyemill['Date/Time (PST)'] = pd.to_datetime(tyemill['Date/Time (PST)'])
tyemill['date'] = pd.to_datetime(tyemill['date'])

In [9]:
# pd.to_datetime(skyline_df['Date/Time (PST)'], '%Y-%m-%d %H:%m') # this could work but it recognized it anyhow

In [10]:
tyemill

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date
0,2019-05-14 07:00:00,12.80,37.21,0.000,4.660,20.34,250.3,2019-05-14
1,2019-05-14 06:00:00,12.84,44.51,0.000,4.046,10.11,278.6,2019-05-14
2,2019-05-14 05:00:00,12.94,43.41,0.723,7.701,15.02,272.7,2019-05-14
3,2019-05-14 04:00:00,13.57,43.25,4.340,10.050,18.87,271.9,2019-05-14
4,2019-05-14 03:00:00,12.64,42.71,3.288,9.440,17.34,275.2,2019-05-14
5,2019-05-14 02:00:00,12.66,43.60,3.529,10.060,16.29,274.3,2019-05-14
6,2019-05-14 01:00:00,12.66,43.74,5.305,12.570,20.67,274.4,2019-05-14
7,2019-05-14 00:00:00,12.67,43.52,2.521,10.580,17.29,274.9,2019-05-14
8,2019-05-13 23:00:00,12.69,44.04,0.745,4.236,10.85,307.3,2019-05-13
9,2019-05-13 22:00:00,12.71,44.01,1.228,5.447,13.37,295.3,2019-05-13


In [11]:
tyemill.isna().sum() #yay, no missing values

Date/Time (PST)             0
Battery Voltage (v)         0
Temperature (deg F)         0
Wind Speed Minimum (mph)    0
Wind Speed Average (mph)    0
Wind Speed Maximum (mph)    0
Wind Direction (deg.)       0
date                        0
dtype: int64

In [12]:
tyemill.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 8 columns):
Date/Time (PST)             3200 non-null datetime64[ns]
Battery Voltage (v)         3200 non-null float64
Temperature (deg F)         3200 non-null float64
Wind Speed Minimum (mph)    3200 non-null float64
Wind Speed Average (mph)    3200 non-null float64
Wind Speed Maximum (mph)    3200 non-null float64
Wind Direction (deg.)       3200 non-null float64
date                        3200 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(6)
memory usage: 200.1 KB


In [13]:
type(tyemill["Date/Time (PST)"][0])

pandas._libs.tslibs.timestamps.Timestamp

In [14]:
tyemill.describe()

,Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.)
count,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000
mean,12.738959,29.232920,1.554205,6.020345,12.275843,197.364617
std,0.225137,10.940916,2.358804,4.425962,7.453986,95.194453
min,12.490000,2.800000,0.000000,0.000000,0.000000,0.000000
25%,12.600000,21.267500,0.000000,3.072750,7.277000,96.775000
50%,12.670000,29.025000,0.219000,4.684000,10.370000,262.100000
75%,12.780000,36.145000,2.455000,7.751750,15.210000,278.000000
max,13.610000,65.950000,17.820000,28.250000,51.990000,358.400000


## Looks like there are negative values for the 24' snow reading, lets explore this after the merge to the avalance-danger table

In [15]:
brooks_precip = pd.read_csv('csv_files/StevensPass-BrooksPrecipitation_4800_feet_2019.csv'
                           )

In [16]:
brooks_precip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3843 entries, 0 to 3842
Data columns (total 7 columns):
Date/Time (PST)          3843 non-null object
Battery Voltage (v)      3843 non-null float64
Precipitation (")        3843 non-null float64
24 Hour Snow (")         3843 non-null float64
Total Snow Depth (")     3843 non-null float64
Temperature (deg F)      1142 non-null float64
Relative Humidity (%)    1142 non-null float64
dtypes: float64(6), object(1)
memory usage: 210.2+ KB


In [17]:
brooks_precip.describe()

,Battery Voltage (v),"Precipitation ("")","24 Hour Snow ("")","Total Snow Depth ("")",Temperature (deg F),Relative Humidity (%)
count,3843.000000,3843.000000,3843.000000,3843.000000,1142.000000,1142.000000
mean,13.193882,0.004385,13.207853,77.578427,43.338555,74.241953
std,0.688588,0.014965,32.502386,32.245231,8.735684,21.295240
min,9.020000,0.000000,-298.800000,-2.899000,22.810000,20.510000
25%,13.015000,0.000000,0.310000,71.120000,37.177500,57.862500
50%,13.220000,0.000000,0.521000,87.200000,42.300000,76.395000
75%,13.650000,0.000000,28.175000,96.700000,48.612500,94.375000
max,14.990000,0.280000,265.600000,220.500000,67.090000,100.000000


In [18]:
brooks_precip.columns #need to get rid of the quotes for inches, rename columns

Index(['Date/Time (PST)', 'Battery Voltage (v)', 'Precipitation (")',
       '24 Hour Snow (")', 'Total Snow Depth (")', 'Temperature (deg F)',
       'Relative Humidity (%)'],
      dtype='object')

In [19]:
brooks_precip.columns=['Date/Time (PST)', 'Battery Voltage (v)', 'Precipitation (in)',
       '24 Hour Snow (in)', 'Total Snow Depth (in)', 'Temperature (deg F)',
       'Relative Humidity (%)']
brooks_precip.tail()

,Date/Time (PST),Battery Voltage (v),Precipitation (in),24 Hour Snow (in),Total Snow Depth (in),Temperature (deg F),Relative Humidity (%)
3838,2019-01-01 04:00,12.99,0.0,0.459,78.13,NaN,NaN
3839,2019-01-01 03:00,13.00,0.0,0.392,77.73,NaN,NaN
3840,2019-01-01 02:00,13.03,0.0,0.459,77.73,NaN,NaN
3841,2019-01-01 01:00,13.09,0.0,0.459,77.73,NaN,NaN
3842,2019-01-01 00:00,13.02,0.0,0.459,77.46,NaN,NaN


In [20]:
brooks_precip['Date/Time (PST)'] = pd.to_datetime(brooks_precip['Date/Time (PST)'])

In [28]:
brooks_precip['Date/Time (PST)'][0], tyemill['Date/Time (PST)'][0]

(Timestamp('2019-06-10 06:00:00'), Timestamp('2019-05-14 07:00:00'))

In [22]:
brooks_precip.head()

,Date/Time (PST),Battery Voltage (v),Precipitation (in),24 Hour Snow (in),Total Snow Depth (in),Temperature (deg F),Relative Humidity (%)
0,2019-06-10 06:00:00,9.07,0.0,65.14,7.095,50.43,65.03
1,2019-06-10 05:00:00,9.10,0.0,61.92,6.186,48.48,64.53
2,2019-06-10 04:00:00,9.39,0.0,61.19,4.693,46.83,65.80
3,2019-06-10 03:00:00,9.45,0.0,62.06,5.506,47.01,61.67
4,2019-06-10 02:00:00,9.66,0.0,61.93,5.490,46.15,56.45


## I only want the 24 snow, snow depth, and rel humidity

In [29]:
brooks_snow_hum =  brooks_precip[['24 Hour Snow (in)','Total Snow Depth (in)','Relative Humidity (%)','Date/Time (PST)']]

In [30]:
merged_df =pd.merge(tyemill,brooks_snow_hum,how='left',on='Date/Time (PST)')

In [31]:
merged_df.head()

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%)
0,2019-05-14 07:00:00,12.80,37.21,0.000,4.660,20.34,250.3,2019-05-14,11.73,40.14,89.30
1,2019-05-14 06:00:00,12.84,44.51,0.000,4.046,10.11,278.6,2019-05-14,13.34,39.53,74.99
2,2019-05-14 05:00:00,12.94,43.41,0.723,7.701,15.02,272.7,2019-05-14,13.34,39.27,69.16
3,2019-05-14 04:00:00,13.57,43.25,4.340,10.050,18.87,271.9,2019-05-14,14.68,40.21,58.96
4,2019-05-14 03:00:00,12.64,42.71,3.288,9.440,17.34,275.2,2019-05-14,13.95,40.47,62.37


In [32]:
# Now to add in the avalanche stuff

In [33]:
stevens_avalanches = pd.read_json('stevens_avalanches.json')

In [26]:
stevens_avalanches.head()

,area,danger_above_treeline,danger_below_treeline,danger_near_treeline,date
0,Stevens Pass,Moderate,Moderate,Moderate,2018-12-31
1,Stevens Pass,Considerable,Moderate,Considerable,2019-01-22
11,Stevens Pass,Low,Low,Low,2019-01-16
12,Stevens Pass,Low,Low,Low,2019-01-15
13,Stevens Pass,Moderate,Low,Moderate,2019-01-14


In [34]:
second_merge =pd.merge(merged_df,stevens_avalanches,how='left',on='date')
second_merge.head()

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),area,danger_above_treeline,danger_below_treeline,danger_near_treeline
0,2019-05-14 07:00:00,12.80,37.21,0.000,4.660,20.34,250.3,2019-05-14,11.73,40.14,89.30,NaN,NaN,NaN,NaN
1,2019-05-14 06:00:00,12.84,44.51,0.000,4.046,10.11,278.6,2019-05-14,13.34,39.53,74.99,NaN,NaN,NaN,NaN
2,2019-05-14 05:00:00,12.94,43.41,0.723,7.701,15.02,272.7,2019-05-14,13.34,39.27,69.16,NaN,NaN,NaN,NaN
3,2019-05-14 04:00:00,13.57,43.25,4.340,10.050,18.87,271.9,2019-05-14,14.68,40.21,58.96,NaN,NaN,NaN,NaN
4,2019-05-14 03:00:00,12.64,42.71,3.288,9.440,17.34,275.2,2019-05-14,13.95,40.47,62.37,NaN,NaN,NaN,NaN


In [55]:
second_merge.tail()
##

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),area,danger_above_treeline,danger_below_treeline,danger_near_treeline
3219,2019-01-01 04:00:00,13.51,21.48,2.872,10.020,21.13,258.7,2019-01-01,0.459,78.13,NaN,Stevens Pass,Moderate,Low,Moderate
3220,2019-01-01 03:00:00,12.54,20.71,3.946,9.600,17.80,262.4,2019-01-01,0.392,77.73,NaN,Stevens Pass,Moderate,Low,Moderate
3221,2019-01-01 02:00:00,12.55,21.19,1.644,9.110,14.71,270.7,2019-01-01,0.459,77.73,NaN,Stevens Pass,Moderate,Low,Moderate
3222,2019-01-01 01:00:00,12.56,22.40,2.214,7.054,11.49,262.1,2019-01-01,0.459,77.73,NaN,Stevens Pass,Moderate,Low,Moderate
3223,2019-01-01 00:00:00,12.56,22.31,1.688,5.976,14.45,235.4,2019-01-01,0.459,77.46,NaN,Stevens Pass,Moderate,Low,Moderate


In [37]:
second_merge['danger_above_treeline'][3219]

'Moderate'

In [38]:
second_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3224 entries, 0 to 3223
Data columns (total 15 columns):
Date/Time (PST)             3224 non-null datetime64[ns]
Battery Voltage (v)         3224 non-null float64
Temperature (deg F)         3224 non-null float64
Wind Speed Minimum (mph)    3224 non-null float64
Wind Speed Average (mph)    3224 non-null float64
Wind Speed Maximum (mph)    3224 non-null float64
Wind Direction (deg.)       3224 non-null float64
date                        3224 non-null datetime64[ns]
24 Hour Snow (in)           3223 non-null float64
Total Snow Depth (in)       3223 non-null float64
Relative Humidity (%)       498 non-null float64
area                        768 non-null object
danger_above_treeline       768 non-null object
danger_below_treeline       768 non-null object
danger_near_treeline        768 non-null object
dtypes: datetime64[ns](2), float64(9), object(4)
memory usage: 563.0+ KB


### now, to just grab the month of january

In [41]:
mask = (second_merge['date'] < ('2019-02-01'))

In [42]:
january = second_merge.loc[mask]

In [43]:
january.info() 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 2456 to 3223
Data columns (total 15 columns):
Date/Time (PST)             768 non-null datetime64[ns]
Battery Voltage (v)         768 non-null float64
Temperature (deg F)         768 non-null float64
Wind Speed Minimum (mph)    768 non-null float64
Wind Speed Average (mph)    768 non-null float64
Wind Speed Maximum (mph)    768 non-null float64
Wind Direction (deg.)       768 non-null float64
date                        768 non-null datetime64[ns]
24 Hour Snow (in)           768 non-null float64
Total Snow Depth (in)       768 non-null float64
Relative Humidity (%)       0 non-null float64
area                        768 non-null object
danger_above_treeline       768 non-null object
danger_below_treeline       768 non-null object
danger_near_treeline        768 non-null object
dtypes: datetime64[ns](2), float64(9), object(4)
memory usage: 96.0+ KB


## I need to convert the string avalanche levels to numbers, lets make a dictionary and then map it to the columns

In [44]:
type(january['danger_above_treeline'][2456])

str

In [45]:
def convert_avy_str_to_num(word):
    if word == 'Low':
        return 1
    elif word == 'Moderate':
        return 2
    elif word == 'Considerable':
        return 3
    elif word == 'High':
        return 4
    elif word == 'Extreme':
        return 5

In [46]:
january.columns

Index(['Date/Time (PST)', 'Battery Voltage (v)', 'Temperature (deg F)',
       'Wind Speed Minimum (mph)', 'Wind Speed Average (mph)',
       'Wind Speed Maximum (mph)', 'Wind Direction (deg.)', 'date',
       '24 Hour Snow (in)', 'Total Snow Depth (in)', 'Relative Humidity (%)',
       'area', 'danger_above_treeline', 'danger_below_treeline',
       'danger_near_treeline'],
      dtype='object')

In [47]:
january[['danger_above_treeline', 'danger_below_treeline','danger_near_treeline']] = january[['danger_above_treeline',
                                                                                              'danger_below_treeline',
                                                                                              'danger_near_treeline']].applymap(convert_avy_str_to_num)



/Users/krex/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [48]:
january.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 2456 to 3223
Data columns (total 15 columns):
Date/Time (PST)             768 non-null datetime64[ns]
Battery Voltage (v)         768 non-null float64
Temperature (deg F)         768 non-null float64
Wind Speed Minimum (mph)    768 non-null float64
Wind Speed Average (mph)    768 non-null float64
Wind Speed Maximum (mph)    768 non-null float64
Wind Direction (deg.)       768 non-null float64
date                        768 non-null datetime64[ns]
24 Hour Snow (in)           768 non-null float64
Total Snow Depth (in)       768 non-null float64
Relative Humidity (%)       0 non-null float64
area                        768 non-null object
danger_above_treeline       768 non-null int64
danger_below_treeline       768 non-null int64
danger_near_treeline        768 non-null int64
dtypes: datetime64[ns](2), float64(9), int64(3), object(1)
memory usage: 116.0+ KB


In [50]:
january.head(1)

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),area,danger_above_treeline,danger_below_treeline,danger_near_treeline
2456,2019-01-31 23:00:00,13.53,40.28,0.0,1.225,4.494,279.8,2019-01-31,0.448,86.5,NaN,Stevens Pass,1,1,1


In [51]:
january.describe() 

,Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),danger_above_treeline,danger_below_treeline,danger_near_treeline
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,0.0,768.000000,768.000000,768.000000
mean,12.747096,27.758099,2.193491,6.345531,12.237727,180.079362,1.638430,83.524857,NaN,2.312500,1.718750,2.062500
std,0.226518,6.491934,2.780953,4.162471,6.816254,93.162920,4.578625,7.632468,NaN,0.917027,0.760198,0.747878
min,12.530000,15.450000,0.000000,0.000000,0.000000,0.000000,0.071000,68.590000,NaN,1.000000,1.000000,1.000000
25%,12.610000,23.130000,0.000000,3.547250,8.062500,96.850000,0.341000,79.240000,NaN,2.000000,1.000000,1.750000
50%,12.680000,27.290000,1.085000,5.412500,11.030000,186.750000,0.459000,83.800000,NaN,2.000000,2.000000,2.000000
75%,12.790000,30.797500,3.595000,8.305000,14.585000,270.600000,1.259750,88.300000,NaN,3.000000,2.000000,3.000000
max,13.610000,46.230000,17.820000,26.230000,51.990000,334.100000,46.760000,204.500000,NaN,4.000000,3.000000,3.000000


In [52]:
january['Date/Time (PST)'] = january['Date/Time (PST)'].map(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

/Users/krex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
january.head(2)

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),area,danger_above_treeline,danger_below_treeline,danger_near_treeline
2456,2019-01-31 23:00:00,13.53,40.28,0.0,1.225,4.494,279.8,2019-01-31,0.448,86.5,NaN,Stevens Pass,1,1,1
2457,2019-01-31 22:00:00,12.58,39.91,0.0,3.626,6.905,108.3,2019-01-31,0.249,86.2,NaN,Stevens Pass,1,1,1


In [54]:
january.to_json('stevens_jan_19.json')

In [47]:
january2 = merged_df.loc[mask]
january2.head(1)

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,area,danger_above_treeline,danger_below_treeline,danger_near_treeline
2456,2019-01-31 23:00:00,13.53,40.28,0.0,1.225,4.494,279.8,2019-01-31,Stevens Pass,Low,Low,Low


In [56]:
january2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 2456 to 3223
Data columns (total 12 columns):
Date/Time (PST)             768 non-null datetime64[ns]
Battery Voltage (v)         768 non-null float64
Temperature (deg F)         768 non-null float64
Wind Speed Minimum (mph)    768 non-null float64
Wind Speed Average (mph)    768 non-null float64
Wind Speed Maximum (mph)    768 non-null float64
Wind Direction (deg.)       768 non-null float64
date                        768 non-null datetime64[ns]
area                        768 non-null object
danger_above_treeline       768 non-null object
danger_below_treeline       768 non-null object
danger_near_treeline        768 non-null object
dtypes: datetime64[ns](2), float64(6), object(4)
memory usage: 98.0+ KB


In [57]:
january2['Date/Time (PST)'] = january2['Date/Time (PST)'].map(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

/Users/krex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
